In [42]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
import os

template = PromptTemplate.from_template("What is the distance between {country1} and {country2}?")

chat = ChatOpenAI(
  temperature=0.1,
  openai_api_key=os.getenv("OPEN_API_KEY"),
)

prompt = template.format(country1="Mexico", country2="Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [43]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
  SystemMessage(content="You are a geography expert. And you only reply in Italian"),
  AIMessage(content="Ciao, mi chiamo Paolo"),
  HumanMessage(content="What is the distance between the Mexico and Thailand? Also what is your name?"),
]

chat.predict_messages(messages)

AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

template = ChatPromptTemplate.from_messages(
    [
      ("system", "You are a geography expert. And you only reply in {language}."),
      ("ai", "Ciao, mi chiamo {name}!"),
      ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [15]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
  def parse(self, text):
    items = text.strip().split(",")
    return list(map(str.strip, items))
  
  
p = CommaOutputParser()

p.parse("Hello, my name, is John")

['Hello', 'my name', 'is John']

In [34]:
template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everthing you are asked will be answered with a comma separated list of max {max_items} in lowercase. DON'T reply with anything else."),
  ("human", "{question}"),
])
                                            
prompt = template.format_messages(
  max_items=10,
  question="What are the colors?",
)

result = chat.predict_messages(prompt)
p = CommaOutputParser()

p.parse(result.content)

red, blue, green, yellow, orange, purple, pink, black, white, brown

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [35]:
chain = template | chat | CommaOutputParser()

chain.invoke({
  "max_items": 5,
  "question": "What are the pokemons?",
})

pikachu, charmander, bulbasaur, squirtle, jigglypuff

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
import os

chat = ChatOpenAI(
  temperature = 0.1,
  streaming = True,
  callbacks=[StreamingStdOutCallbackHandler()],
  openai_api_key=os.getenv("OPEN_API_KEY")
)

chef_template = ChatPromptTemplate.from_messages([
  ("system", "You are a world-class international chef. You are asked to create a recipe with easy to find ingredients."),
  ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_template | chat

In [41]:
## 풀스택 GPT 3.4 (20241209)

veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it. "),
  ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
  "cuisine": "Indian",
})

Great choice! How about trying a classic Indian dish like Chicken Tikka Masala? It's a flavorful and aromatic dish that is sure to impress your friends and family. Here's a simple recipe for you to try:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) diced tomatoes
- 2 tablespoons tomato paste
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- 1/2 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 1/2 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and marinate for at least 1 ho

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. Tofu: Use firm or extra-firm tofu for this recipe. Drain the tofu and press it to remove excess water. Cut the tofu into bite-sized cubes and follow the marinating instructions in the recipe. Tofu absorbs flavors well, so marinating it for a longer time will enhance the taste. Instead of baking, you can pan-fry the marinated tofu until it's golden brown and slightly crispy.\n\n2. Paneer: Paneer is a type of Indian cheese that holds its shape well when cooked. You can either make your own paneer at home or buy it from the store. Cut the paneer into bite-sized cubes and follow the marinating instructions in the recipe. Since paneer is already firm, you can skip the baking step and directly add the marinated paneer to the sauce in the skillet. Cook it gently to allow the flavors to 

In [6]:
## 풀스택 GPT 4.1 (20241210)

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
import os

chat = ChatOpenAI(
  temperature = 0.1,
  streaming = True,
  callbacks=[StreamingStdOutCallbackHandler()],
  openai_api_key=os.getenv("OPEN_API_KEY")
)

examples = [
  {"question": "What do you know about France?",
   "answer": """
   Here is what I know
   Capital: Paris
   Language: French
   Currency: Euro
   Food: French Fries
   """
  },
  {"question": "What do you know about Germany?",
   "answer": """
   I know this:
   Capital: Berlin
   Language: German
   Currency: Euro
   Food: Bratwurst
   """
  },
  {"question": "What do you know about Italy?",
   "answer": """
   Here is what I know
   Capital: Rome
   Language: Italian
   Currency: Euro
   Food: Pizza
   """
  },
]

# same variables with examples
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
  suffix="Human: What do you know about {country}?",
  input_variables=["country"],
)

chain = prompt | chat

chain.invoke({
  "country": "Turkey"
})


AI: 
   Here is what I know:
   Capital: Ankara
   Language: Turkish
   Currency: Turkish Lira
   Food: Kebab

AIMessageChunk(content='AI: \n   Here is what I know:\n   Capital: Ankara\n   Language: Turkish\n   Currency: Turkish Lira\n   Food: Kebab')

In [11]:
#풀스택 GPT 4.2 (20241210)

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
import os

chat = ChatOpenAI(
  temperature = 0.1,
  streaming = True,
  callbacks=[StreamingStdOutCallbackHandler()],
  openai_api_key=os.getenv("OPEN_API_KEY")
)

examples = [
  {"country": "France",
   "answer": """
   Here is what I know
   Capital: Paris
   Language: French
   Currency: Euro
   Food: French Fries
   """
  },
  {"country": "Germany",
   "answer": """
   I know this:
   Capital: Berlin
   Language: German
   Currency: Euro
   Food: Bratwurst
   """
  },
  {"country": "Italy",
   "answer": """
   Here is what I know
   Capital: Rome
   Language: Italian
   Currency: Euro
   Food: Pizza
   """
  },
]

example_prompt = ChatPromptTemplate.from_messages([
  ("human", "What do you know about {country}"),
  ("ai", "{answer}"),
])

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a geography expert. You have to give me short answer."),
  example_prompt,
  ("human", "What do you know about {country}?"),
])

chain = final_prompt | chat

chain.invoke({
  "country": "Thailand"
})



   Here is what I know
   Capital: Bangkok
   Language: Thai
   Currency: Thai Baht
   Food: Pad Thai

AIMessageChunk(content='\n   Here is what I know\n   Capital: Bangkok\n   Language: Thai\n   Currency: Thai Baht\n   Food: Pad Thai')

In [16]:
#풀스택 GPT 4.3 (20241210)

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
import os
    

chat = ChatOpenAI(
  temperature = 0.1,
  streaming = True,
  callbacks=[StreamingStdOutCallbackHandler()],
  openai_api_key=os.getenv("OPEN_API_KEY")
)

examples = [
  {"question": "What do you know about France?",
   "answer": """
   Here is what I know
   Capital: Paris
   Language: French
   Currency: Euro
   Food: French Fries
   """
  },
  {"question": "What do you know about Germany?",
   "answer": """
   I know this:
   Capital: Berlin
   Language: German
   Currency: Euro
   Food: Bratwurst
   """
  },
  {"question": "What do you know about Italy?",
   "answer": """
   Here is what I know
   Capital: Rome
   Language: Italian
   Currency: Euro
   Food: Pizza
   """
  },
]

class RandomExampleSelector(BaseExampleSelector):
  
  def __init__(self, examples):
    self.examples = examples
    
  def add_example(self, example):
    self.examples.append(example)
  
  def select_examples(self, input_variables):
    from random import choice
    return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
  examples=examples,
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?",
  input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI: \n   Here is what I know\n   Capital: Rome\n   Language: Italian\n   Currency: Euro\n   Food: Pizza\n   \n\nHuman: What do you know about Brazil?'

In [18]:
#풀스택 GPT 4.4 (20241210)

import os
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate
# from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
  temperature = 0.1,
  streaming = True,
  callbacks=[StreamingStdOutCallbackHandler()],
  openai_api_key=os.getenv("OPEN_API_KEY")
)

# prompt.format(country="Germany")

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
  final_prompt=final, 
  pipeline_prompts=prompts
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea in me belly! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea in me belly! Arrr!")

In [23]:
#풀스택 GPT 4.5 (20241210)

import os
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
  temperature = 0.1,
  # streaming = True,
  # callbacks=[StreamingStdOutCallbackHandler()],
  openai_api_key=os.getenv("OPEN_API_KEY")
)

chat.predict("How do you make Italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make Italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [5.22s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEn

In [33]:
#풀스택 GPT 4.5 (20241210)

import os
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

# chat = OpenAI(
#   temperature = 0.1,
#   openai_api_key=os.getenv("OPEN_API_KEY"),
#   max_tokens=450,
#   model="gpt-3.5-turbo-16k"
# )

# chat.save("model.json")
chat=load_llm("model.json")

chat


/Users/kpkim/Documents/Fullstack GPT/venv/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


ValidationError: 1 validation error for OpenAIChat
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)